<a href="https://colab.research.google.com/github/leito77/Nanocables/blob/main/rotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Notebook associated with the article "Supramolecular Polymers of Amino-triazines vs. Amino-pyrimidines: How key Interactions Control their Thermodynamic Stability"**



## **Set working directory to save images**

In [3]:
#@title **Load Google drive**
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [17]:
#@title **Making folders to save outputs**
import os
folder_name = 'save_nanocable_data'

# Especifica la ruta donde se debe crear la carpeta
path_folder = '/content/drive/My Drive/'

work_folder=os.path.join(path_folder, folder_name)

# Utiliza la función os.makedirs() para crear la carpeta
try:
    os.makedirs(os.path.join(work_folder))
    print('Folder created successfully')
except OSError as e:
    print(f'Error creating folder: {e}')


Error creating folder: [Errno 17] File exists: '/content/drive/My Drive/save_nanocable_data'


## **Downloading necesary data**

In [2]:
!pip install py3Dmol

In [3]:
#@title **Cloning GitHub repository**
!git clone https://github.com/leito77/Nanocables.git

fatal: destination path 'Nanocables' already exists and is not an empty directory.


In [4]:
#@title **Open folder**
%cd Nanocables

/content/Nanocables


In [5]:
#@title **list of files**
%ls

condacolab_install.log  README.md               tapas_12.pdb
Extensor.ipynb          salida.txt              tapas_CA_rosetax1_nanocable.pdb
LICENSE                 tapas_12_nanocable.pdb  tapas_CA_rosetax1.pdb


In [2]:
from IPython.utils import io
import tqdm.notebook
import os

total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        !pip install -q condacolab
        import condacolab
        condacolab.install()
        pbar.update(10)

        import sys
        sys.path.append('/usr/local/lib/python3.7/site-packages/')
        pbar.update(20)

        # Install PyMOL
        %shell mamba install -c schrodinger pymol-bundle --yes

        pbar.update(90)

        # Mount Google Drive and upload your PyMOL license
        #from google.colab import drive
        #drive.mount("/content/drive")
        #%shell cp ./drive/MyDrive/Colab\ Notebooks/license.lic .
        pbar.update(100)


  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
#@title **import modules**
import py3Dmol
from pymol import cmd
from IPython.display import Image
import pandas as pd

In [7]:
#@title **Load files**
name_file="tapas_CA_rosetax1_nanocable.pdb"  #CHANGE NAME FILE
name_base = name_file.split(".")[0]

In [8]:
# Reads the PDB file and separates columns based on spaces
data = []
with open(name_file, 'r') as file:
    for line in file:
        if line.startswith('ATOM'):
            parts = line.split()
            atom = parts[0]
            atom_number = parts[1]
            atom_type = parts[2]
            atom_name = parts[3]
            residue_number = (parts[4])
            x = float(parts[5])
            y = float(parts[6])
            z = float(parts[7])
            occupancy = (parts[8])

            data.append([atom,atom_number,atom_type, atom_name, residue_number, x, y, z,occupancy])

# Create a Pandas DataFrame with the obtained data
df = pd.DataFrame(data, columns=["atom", "atom_number",'Atom Type', 'Atom Name', 'Residue Number', 'X', 'Y', 'Z',"occupancy"])
df

,atom,atom_number,Atom Type,Atom Name,Residue Number,X,Y,Z,occupancy
0,ATOM,1,O1,CAA,1,-3.657,6.320,0.016,0.00
1,ATOM,2,C1,CAA,1,-2.507,5.925,0.044,0.00
2,ATOM,3,N1,CAA,1,-2.157,4.593,0.094,0.00
3,ATOM,4,N2,CAA,1,-1.428,6.798,0.038,0.00
4,ATOM,5,C2,CAA,1,-0.861,4.125,0.071,0.00
...,...,...,...,...,...,...,...,...,...
2275,ATOM,2276,C7,TAP,120,-9.469,-0.680,63.959,0.00
2276,ATOM,2277,H9,TAP,120,-9.054,-1.128,63.033,0.00
2277,ATOM,2278,H10,TAP,120,-9.484,-1.523,64.712,0.00
2278,ATOM,2279,C8,TAP,120,-8.588,0.426,64.498,0.00


In [24]:
#@title **Visualization**
with open(name_file) as ifile:
  system="".join([x for x in ifile])

view = py3Dmol.view(width=600, height=600)
view.addModel(system,"pdb")
# Establecer estilo para ocultar átomos de hidrógeno
view.addStyle({'stick': {}})
#view.addStyle( {'stick': {'colorscheme':'orangeCarbon'}})
#view.addStyle({'resn':"SER499"}, {'stick': {'colorscheme':'orangeCarbon'}})
view.addStyle({"label":{"colorscheme":"greenCarbon"}})
view.zoomTo()
view.show()

# Guardar la imagen
view.png()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
#@title **Load files in PYMOL**
cmd.load(name_file)

1

In [11]:
#@title **Number of molecules**
max_resid_value = df.iloc[-1,4]
max_resid_value

'120'

In [12]:
num_rosettes = int(max_resid_value) / 6 # 6 molecules per rosette
num_rosettes = int(num_rosettes)
num_rosettes

20

In [71]:
#@title **Load files in PYMOL**
cmd.load(name_file)

angle_to_rotate = 60 # change by angle of interest
num_residuos = 6 # 6 molecules per rosette

lst = []

#with open(work_folder+"/"+name_base+"_rotate.pdb", "w") as file:
for i in range(num_rosettes):
    resid_initial = 1 + i*num_residuos
    resid_final = resid_initial + num_residuos - 1
    print(f"t{i+1} range {resid_initial}:{resid_final}")

    cmd.select(f"t{i+1}", f"resid {resid_initial}:{resid_final}")
    cmd.rotate("z",i*angle_to_rotate,f"t{i+1}")
    cmd.save(work_folder+"/"+name_base+"_rotate.pdb")
    print(f"t{i+1} has been rotated {i*angle_to_rotate} degrees")

#for i in range(5):
    #cmd.rotate("z",i*2*angle_to_rotate,f"t{i+1}")
    #cmd.rotate("z",angle_to_rotate,f"t{i+1}")
#cmd.rotate("z",angle_to_rotate,"t1")
#cmd.rotate("z",2*angle_to_rotate,"t2")

    #print(f"t{i+1} has been rotated {i*angle_to_rotate} degrees")

#cmd.save(work_folder+"/"+name_base+"_rotate.pdb")

t1 range 1:6
t1 has been rotated 0 degrees
t2 range 7:12
t2 has been rotated 60 degrees
t3 range 13:18
t3 has been rotated 120 degrees
t4 range 19:24
t4 has been rotated 180 degrees
t5 range 25:30
t5 has been rotated 240 degrees
t6 range 31:36
t6 has been rotated 300 degrees
t7 range 37:42
t7 has been rotated 360 degrees
t8 range 43:48
t8 has been rotated 420 degrees
t9 range 49:54
t9 has been rotated 480 degrees
t10 range 55:60
t10 has been rotated 540 degrees
t11 range 61:66
t11 has been rotated 600 degrees
t12 range 67:72
t12 has been rotated 660 degrees
t13 range 73:78
t13 has been rotated 720 degrees
t14 range 79:84
t14 has been rotated 780 degrees
t15 range 85:90
t15 has been rotated 840 degrees
t16 range 91:96
t16 has been rotated 900 degrees
t17 range 97:102
t17 has been rotated 960 degrees
t18 range 103:108
t18 has been rotated 1020 degrees
t19 range 109:114
t19 has been rotated 1080 degrees
t20 range 115:120
t20 has been rotated 1140 degrees


In [72]:
#@title **Visualization**
with open(work_folder+"/"+name_base+"_rotate.pdb") as ifile:
  system="".join([x for x in ifile])

view = py3Dmol.view(width=600, height=600)
view.addModel(system,"pdb")
# Establecer estilo para ocultar átomos de hidrógeno
view.addStyle({'stick': {}})
#view.addStyle( {'stick': {'colorscheme':'orangeCarbon'}})
#view.addStyle({'resn':"SER499"}, {'stick': {'colorscheme':'orangeCarbon'}})
view.addStyle({"label":{"colorscheme":"greenCarbon"}})
view.zoomTo()
view.show()

# Guardar la imagen
view.png()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.